In [3]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [4]:
import time
import datetime
import math
import tensorflow as tf
import numpy as np
import pathlib
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "false"
os.environ["TF_GPU_ALLOCATOR"] = ""

model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255, input_shape=(64, 64, 3)),

    tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(64, 64, 3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

# model.summary()

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Monte carlo cross validation

data_path = "/tf/data/train"
data_dir = pathlib.Path(data_path)

batch_size = 64
img_height = 64
img_width = 64

all_scores = []
all_histories = []

# Monte carlo cross validation
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir, histogram_freq=1)


for i in range(25):
    seed = math.ceil(time.time())
    train_ds = tf.keras.utils.image_dataset_from_directory(
        data_dir,
        validation_split=0.2,
        subset="training",
        seed=seed,
        image_size=(img_height, img_width),
        batch_size=batch_size)

    val_ds = tf.keras.utils.image_dataset_from_directory(
        data_dir,
        validation_split=0.2,
        subset="validation",
        seed=seed,
        image_size=(img_height, img_width),
        batch_size=batch_size)

    # history = model.fit(train_ds, validation_data=val_ds, epochs=20)
    history = model.fit(train_ds, epochs=10, callbacks=[tensorboard_callback])
    val_mse, val_mae = model.evaluate(val_ds, verbose=0)
    all_histories.append(history)
    all_scores.append(val_mae)

print(np.mean(all_scores))


Found 2400 files belonging to 2 classes.
Using 1920 files for training.
Found 2400 files belonging to 2 classes.
Using 480 files for validation.
Epoch 1/10
30/30 [==============================] - 1s 14ms/step - loss: 0.6201 - accuracy: 0.6562
Epoch 2/10
30/30 [==============================] - 0s 3ms/step - loss: 0.5786 - accuracy: 0.7016
Epoch 3/10
30/30 [==============================] - 0s 4ms/step - loss: 0.5365 - accuracy: 0.7359
Epoch 4/10
30/30 [==============================] - 0s 4ms/step - loss: 0.4942 - accuracy: 0.7573
Epoch 5/10
30/30 [==============================] - 0s 3ms/step - loss: 0.4698 - accuracy: 0.7760
Epoch 6/10
30/30 [==============================] - 0s 4ms/step - loss: 0.4520 - accuracy: 0.7771
Epoch 7/10
30/30 [==============================] - 0s 4ms/step - loss: 0.4219 - accuracy: 0.8078
Epoch 8/10
30/30 [==============================] - 0s 4ms/step - loss: 0.4101 - accuracy: 0.8198
Epoch 9/10
30/30 [==============================] - 0s 4ms/step - loss

In [5]:

np.std(all_scores)

0.04775777882592894

In [6]:
%tensorboard --logdir logs/fit



In [7]:
import matplotlib.pyplot as plt

history = all_histories[0]

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

print("Test Acc: ", acc[-1])
print("Val Acc: ", val_acc[-1])


KeyError: 'val_accuracy'